In [1]:
import pandas as pd
import psycopg2
from datetime import datetime
import xml.etree.ElementTree as ET
import os

db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}


# Define the root directory path
root_directory = r'D:\W\RELEX\RELEX\Future_List_Cost\Inbound_files'
table_stg = "cspoms.LIST_COST_MAINT_STG".upper()
table_main = "cspoms.xxpo_future_list_cost_master".upper()

In [48]:
# Function to process a file
def process_file(file_path, request_id, current_timestamp, cursor):
    print(f"Processing file: {file_path}")
    xml_data = ""  # Variable to store the entire XML data
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # Replace the second occurrence of <TRANSACTIONID> with </TRANSACTIONID>
                line = replace_second_occurrence(line, "<TRANSACTIONID>", "</TRANSACTIONID>")

                # Append the line to the XML data variable
                xml_data += line

                # Extract values using regular expression
#                 print(line)
            except Exception as e:
                print(f"Error processing file <TRANSACTIONID>:")
                print(f"Error: {e}")

    # Parse the entire XML data
    root = ET.fromstring('<root>' + xml_data + '</root>')
    # Now you can further process the XML data as needed
    # Extract and insert data into the table
    rows_count = 0  # Counter for the number of rows
    for future_list in root.findall('.//LIST_COST_MAINT'): 
        fac_item_number = get_element_text(future_list.find('BICEPSITEM'))
        # Function to format dates


        # Example usage
        first_order_date = get_element_text(future_list.find('FIRSTORDERDATE'))
        formatted_first_order_date = format_date(first_order_date)

        last_order_date = get_element_text(future_list.find('LASTORDERDATE'))
        formatted_last_order_date = format_date(last_order_date)

        sell_effective_date = get_element_text(future_list.find('DATESELLEFF'))
        formatted_sell_effective_date = format_date(sell_effective_date)

        future_cost = get_element_text(future_list.find('ITEMLISTFUTURE'))
        mfg_deal_num = get_element_text(future_list.find('DEALMANUFACTURENUMBER'))
        empty_tags = [elem.tag for elem in future_list if elem.text is None]
        # Convert list of tag names to a string (you can customize this based on your needs)
        action_code = ', '.join(empty_tags)
        print(action_code)
        transaction_id = get_element_text(future_list.find('TRANSACTIONID'))

        # Calculate facility_num and ssic based on BICEPS_ITEM length
        cs_fac = None
        cs_item = None
        if len(fac_item_number) == 9:
            cs_fac = fac_item_number[0:3]
            cs_item = fac_item_number[3:9]
        elif len(fac_item_number) == 8:
            cs_fac = fac_item_number[0:2]
            cs_item = fac_item_number[2:8]
        # Print variables to see what is captured in each
        # Print variables to see what is captured in each
        print(f"fac_item_number: {fac_item_number}")
        print(f"cs_fac: {cs_fac}")
        print(f"cs_item: {cs_item}")
        print(f"first_order_date: {first_order_date}")
        print(f"last_order_date: {last_order_date}")
        print(f"future_cost: {future_cost}")
        print(f"sell_effective_date: {sell_effective_date}")
        print(f"mfg_deal_num: {mfg_deal_num}")
        print(f"Empty_tags: {action_code}")
        print(f"Transaction_ID: {transaction_id}")
#         column_names = 
        insert_query = f"""
        INSERT INTO {table_stg} (fac_item_number, ssic ,facility, first_order_date, last_order_date, future_cost,
                                    sell_effective_date,mfg_deal_num,list_cost_type,transaction_id, process_status, created_by, last_updated_by, creation_date, last_update_date, request_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                        """

        cursor.execute(insert_query, (fac_item_number,cs_item, cs_fac, formatted_first_order_date, formatted_last_order_date, future_cost,
                                formatted_sell_effective_date,mfg_deal_num, action_code, transaction_id, 'N',
                                -1, -1, current_timestamp, current_timestamp, request_id))
        rows_count += 1  # Increment the rows counter

    print(f"Processed file: {file_name}, Number of rows inserted: {rows_count}")

    print('#######################################')

In [45]:
def format_date(date_str):
    # Assuming the input date_str is in MMDDYY format
    date_object = datetime.strptime(date_str, '%m%d%y')
    return date_object.strftime('%Y-%m-%d')

In [23]:
def get_element_text(element, default_value=None):
    """Get the text of an XML element or return a default value if the element is not found."""
    return element.text if element is not None else default_value

In [39]:
# Function to replace the second occurrence of a string in each line
def replace_second_occurrence(input_str, search_str, replace_str):
    # Find the first occurrence of the search string
    first_occurrence = input_str.find(search_str)

    # If the search string is found
    if first_occurrence != -1:
        # Find the second occurrence of the search string starting from the position after the first occurrence
        second_occurrence = input_str.find(search_str, first_occurrence + 1)

        # If the second occurrence is found
        if second_occurrence != -1:
            # Replace the second occurrence with the replace string
            output_str = input_str[:second_occurrence] + replace_str + input_str[second_occurrence + len(search_str):]

            return output_str

    # If either the first or second occurrence is not found, return the original string
    return input_str

In [58]:
from datetime import datetime
conn = psycopg2.connect(**db_params)
cur = conn.cursor()
# Establish a database connection
with psycopg2.connect(**db_params) as post_conn:
    with post_conn.cursor() as post_cur:
        # Get the nextval for request_id
        nextval_request_id = "SELECT nextval('cspoms.xxpo_cs_relex_run_id_seq')"
        post_cur.execute(nextval_request_id)
        request_id = post_cur.fetchone()[0]
        post_conn.commit()
        print(f"Request ID: {request_id}")

        # Get the current timestamp
        current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Current Timestamp: {current_timestamp}")

        # Specify the file name pattern
        file_name_pattern = 'FUTRLIST.IN07'

        # Iterate over files in the directory
        for file_name in os.listdir(root_directory):
            if file_name_pattern in file_name:
                # Join the root directory path and the file name
                file_path = os.path.join(root_directory, file_name)

                # Establish a database connection
                with psycopg2.connect(**db_params) as conn:
                    print('db_params :', db_params)
                    print('conn :',conn)
                    with conn.cursor() as cursor:
                        # Process the file
                        process_file(file_path, request_id, current_timestamp, cursor)
                        
                        # Commit the changes to the database
                        conn.commit()
         # Fetch the df for the request_id
        post_cur.execute(f'SELECT * FROM LIST_COST_MAINT_STG WHERE request_id = {request_id}')
        columns = [desc[0] for desc in post_cur.description]
        df = pd.DataFrame(post_cur.fetchall(), columns=columns)

print(f"Records inserted to staging table with request_id: {request_id}")
print("DataFrame for the inserted records:")
# print(df)

#Validating facility_num and ssic with cspoms_item_master table
validate_query = f"""
SELECT a.*
                    FROM cspoms.LIST_COST_MAINT_STG a,
                    cspoms.cspoms_item_master b
                    WHERE '0000'||a.ssic = b.ssic
                    AND a.facility = b.facility_num
                    AND b.inventory_item_status_code IN ('PenDisc', 'Active')
					AND a.process_status in ('N','R')
                    --AND a.request_id = {request_id}
					--AND a.request_id = 43
                """
# cur.execute(update_status_query)
df_validation_result = pd.read_sql_query(validate_query, conn)
print(df_validation_result)

# Identify the records to exclude from the update
print("Columns in df_validation_result:", df_validation_result.columns)
records_to_exclude = set(df_validation_result['fac_item_number'])
print('validated records : ',records_to_exclude)


#taking the stg table data based the current request_id
stg_query = f"""
                select * from cspoms.LIST_COST_MAINT_STG where Process_status IN ('N')
                --AND request_id =  {request_id}
            """
df_stg = pd.read_sql_query(stg_query, conn)
print(df_stg)

# taking 'fac_item_number' is the unique identifier
df_stg['fac_item_number'] = df_stg['fac_item_number'].astype(str)

# Perform set difference to get records to update
records_to_update = set(df_stg['fac_item_number']) - records_to_exclude

# Convert the set to a DataFrame
df_to_update = pd.DataFrame({'fac_item_number': list(records_to_update)})
print('records are not there in item master : ',df_to_update)

# Updating the process_status in the stg table
for _, row in df_to_update.iterrows():
    fac_item_nbr = row['fac_item_number']
    update_query = f"""
        UPDATE cspoms.LIST_COST_MAINT_STG
        SET process_status = 'E'
        --, error_message = 'Record not there in Item Master Table.'
        WHERE 1 = 1
            AND fac_item_number = '{fac_item_nbr}'
            AND process_status = 'N'
    """
    cur.execute(update_query)
    status_na = cur.rowcount
conn.commit()
print(f"===== Records are not there in Item Master(E) : ======, {status_na}")


Request ID: 52
Current Timestamp: 2024-01-03 22:05:04
db_params : {'host': 'gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com', 'database': 'CSPODB', 'port': 5432, 'user': 'cspoms', 'password': 'cspoms', 'sslmode': 'verify-ca', 'sslcert': 'D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem', 'sslkey': 'D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem', 'sslrootcert': 'D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem'}
conn : <connection object at 0x000002C6B39BD480; dsn: 'user=cspoms password=xxx dbname=CSPODB host=gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com port=5432 sslmode=verify-ca sslcert=D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem sslkey=D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem sslrootcert=D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem', closed: 0>
Processing file: D:\W\RELEX\RELEX\Future_List_Cost\Inbound_files\FUTRLIST.IN07.20231205190925690 (MF File)
LISTCOSTADD
fac_item_number: 74150855
cs_fac: 74
cs_item: 150855
first_order_date: 011124
last_or

LISTCOSTADD
fac_item_number: 250494237
cs_fac: 250
cs_item: 494237
first_order_date: 120723
last_order_date: 010624
future_cost: 00024.2600
sell_effective_date: 010724
mfg_deal_num: 120730106ORA2426
Empty_tags: LISTCOSTADD
Transaction_ID: L120623000000000000002
LISTCOSTADD
fac_item_number: 250091739
cs_fac: 250
cs_item: 091739
first_order_date: 030824
last_order_date: 040724
future_cost: 00014.2000
sell_effective_date: 040824
mfg_deal_num: 030840407ORA1420
Empty_tags: LISTCOSTADD
Transaction_ID: L120623000000000000003
LISTCOSTADD
fac_item_number: 250952159
cs_fac: 250
cs_item: 952159
first_order_date: 030824
last_order_date: 040724
future_cost: 00013.5200
sell_effective_date: 040824
mfg_deal_num: 030840407ORA1352
Empty_tags: LISTCOSTADD
Transaction_ID: L120623000000000000004
LISTCOSTADD
fac_item_number: 250494237
cs_fac: 250
cs_item: 494237
first_order_date: 030824
last_order_date: 040724
future_cost: 00022.6500
sell_effective_date: 040824
mfg_deal_num: 030840407ORA2265
Empty_tags: LIS

          fac_item_number                  ssic    facility first_order_date  \
0    74150855              150855                74               2024-01-11   
1    74386343              386343                74               2024-01-11   
2    74572329              572329                74               2024-01-11   
3    74649191              649191                74               2024-01-11   
4    74871449              871449                74               2024-01-11   
..                    ...                   ...         ...              ...   
292  250952159             952159                250              2024-03-08   
293  250494237             494237                250              2024-03-08   
294  250473199             473199                250              2024-03-08   
295  251582038             582038                251              2024-01-05   
296  251580418             580418                251              2024-01-05   

    last_order_date  future_cost sell_e

In [ ]:
with psycopg2.connect(**db_params) as post_conn:
    with post_conn.cursor() as post_cur:
        # Get the nextval for request_id
        Get_fac_mfg_comb = "SELECT "
        post_cur.execute(nextval_request_id)
        request_id = post_cur.fetchone()[0]
        post_conn.commit()
        print(f"Request ID: {request_id}")